In [3]:

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [4]:
ipc = pd.read_excel('planilhas/vendas/vendedores/ipc.xls')
ipc

,codigo,funcionario,ipc
0,1,PEDRINHO BRITO,0.00
1,2,LUIZ ANDRE FERREIRA,1.22
2,3,**ACESSO,0.00
3,4,*VANESSA JULIANI ROWE,0.00
4,5,*ELIAS JUNIOR DE FREITAS SANTOS,0.00
...,...,...,...
112,117,MAURICIO DOS SANTOS DA SILVA,0.00
113,118,MARCIA PORTO FERNANDES,0.00
114,119,BOT,0.00
115,121,ARTHUR DIAS DOS SANTOS,0.00


In [10]:
ipc = ipc.query("codigo == 10")
ipc['ipc']

9    1.96
Name: ipc, dtype: float64

In [2]:


df_perfumaria = pd.read_excel('planilhas/comissao_062024_perfumaria.xls', header=9, usecols=('B,C,I'))[0:-2]
df_perfumaria.rename(columns={'Código':'codigo','Vendedor':'vendedor','Base Comissão':'perfumaria'}, inplace=True)
df_gen_sim = pd.read_excel('planilhas/comissao_062024_gen_sim.xls', header=9, usecols=('B,C,I'))[0:-2]
df_gen_sim.rename(columns={'Código':'codigo','Vendedor':'vendedor','Base Comissão':'gen_sim'}, inplace=True)
df_bonificados = pd.read_excel('planilhas/bonificados_062024.xls')
df = pd.merge(df_perfumaria,df_gen_sim, on=['codigo','vendedor'])
df = pd.merge(df,df_bonificados, on=['codigo','vendedor'], how='outer')
df = df.fillna(0)
df_metas = pd.read_excel('planilhas/metas_balconistas_062024.xlsx')
df_metas.rename(columns={
    'Código':'codigo',
    'Vendedor':'vendedor',
    'Valor previsto':'meta',
    'Valor realizado':'venda_total',
    },
    inplace=True)
df = pd.merge(df,df_metas, on=['codigo','vendedor'])
df['meta_atingida'] = np.where(
    df['venda_total'] > df['meta'],'sim','nao'
    )

df['comissao'] = np.where(df['codigo'] == 84,400,np.where(
    df['meta_atingida'] == 'sim', round((df['gen_sim']*0.05)+(df['perfumaria']*0.03)+(df['valor_bonificados']),2),round((df['gen_sim']*0.03)+(df['perfumaria']*0.03)+df['valor_bonificados'],2)
))
df_lideres = pd.read_excel('planilhas/metas_lideres_062024.xlsx')
df = pd.merge(df,df_lideres, on=['codigo','vendedor'], how='outer')
df = df[df['cargo'] != 'LIDER']
df.drop(columns={'cargo','cod_loja','meta_loja','venda_loja'}, inplace=True)
df['vendedor'] = df['vendedor'].str.strip()

df_lideres['meta_atingida'] = np.where(
    df_lideres['venda_loja'] > df_lideres['meta_loja'],'sim','nao'
    )
df_lideres['% realizado'] = round(df_lideres['venda_loja']/df_lideres['meta_loja']*100,1)
df_lideres['comissao'] = np.where(
    df_lideres['meta_atingida'] == 'sim', round((df_lideres['venda_loja']*0.005),2),round((df_lideres['venda_loja']*0.0025),2))

total_comissoes_vendedores = round(df['comissao'].sum(),2)
total_comissoes_lideres = round(df_lideres['comissao'].sum(),2)
total_comissoes = total_comissoes_vendedores+total_comissoes_lideres
venda_total = 862579.53

disclaimer = [
    f'O valor total pago em comissões no mês corrente referente ao mês de Maio/2024 foi de {total_comissoes} reais',
    f'Os vendedores receberam {total_comissoes_vendedores} reais e os líderes {total_comissoes_lideres} reais',
    f'Considerando a venda total da rede Shopfarma de R$ {venda_total}, a parcela paga em comissões representa {round(total_comissoes/venda_total*100,2)}%',
]


In [3]:
df

,codigo,vendedor,perfumaria,gen_sim,bonificados,valor_bonificados,meta,venda_total,% realizado,meta_atingida,comissao
1,10,OTAVIO ALVES DE ALMEIDA,0.00,24075.04,4.0,20.0,50000.0,65088.00,130.176000,sim,1223.75
3,15,RENATO NUNES,0.00,13927.41,0.0,0.0,32000.0,24665.15,77.078594,nao,417.82
5,38,CARMEN LUCIA SILVEIRA GONCALVES,0.00,21740.42,3.0,15.0,42000.0,44414.52,105.748857,sim,1102.02
8,45,MICHELLE ENCK,2813.53,1734.91,1.0,5.0,25000.0,7481.24,29.924960,nao,141.45
9,52,DIEGO NUNES,0.00,13290.52,4.0,20.0,32000.0,24650.55,77.032969,nao,418.72
10,65,MIRELLY DE CASSIA PALACIOS FERREIRA,0.00,2234.76,1.0,5.0,30000.0,10099.55,33.665167,nao,72.04
12,67,RICARDO RAMOS CASSOL,0.00,8634.91,3.0,15.0,30000.0,23321.34,77.737800,nao,274.05
13,70,DIRCEMARA SOARES CARVALHO,0.00,8047.20,12.0,60.0,33200.0,28753.17,86.605934,nao,301.42
14,72,GENECI MARIA DE CASTRO SILVEIRA,0.00,21759.31,9.0,45.0,56000.0,56478.00,100.853571,sim,1132.97
15,73,MARIELE DE SOUZA LEITE,0.00,9120.10,2.0,10.0,36000.0,19785.18,54.958833,nao,283.60


In [4]:
df_lideres

,codigo,cargo,vendedor,cod_loja,meta_loja,venda_loja,meta_atingida,% realizado,comissao
0,66,LIDER,ELAINE COELHO,1,171000,163196.62,nao,95.4,407.99
1,42,LIDER,ERINEU FRANCISCO CORDEIRO,4,80000,71940.74,nao,89.9,179.85
2,33,LIDER,SHIRLEY MARLENE SANTOS,5,120000,110680.91,nao,92.2,276.70
3,93,LIDER,FLAVIA BATISTA VILACA,7,110000,98867.32,nao,89.9,247.17
4,9,LIDER,SILVANIA MATOS-,8,145000,138309.48,nao,95.4,345.77
5,11,LIDER,SHANDRICA SOARES RODRIGUES,9,50000,40899.64,nao,81.8,102.25
6,43,LIDER,EDSON ROSARIO,10,200000,180332.01,nao,90.2,450.83


In [5]:
print(disclaimer[0])
print(disclaimer[1])
print(disclaimer[2])

O valor total pago em comissões no mês corrente referente ao mês de Maio/2024 foi de 13393.57 reais
Os vendedores receberam 11383.01 reais e os líderes 2010.56 reais
Considerando a venda total da rede Shopfarma de R$ 862579.53, a parcela paga em comissões representa 1.55%
